In [85]:
import pandas as pd
import options_analysis as oa
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import datetime
import numpy as np
import scipy as sc
import mariadb as mdb

In [87]:
d = pd.read_csv("E:\\GitHub\\coursera\\courses\\py portfolio\\notebooks_and_codem01_v02\\nb\\data\\aapl_options.csv",
                header='infer', sep=';', parse_dates=True)

In [88]:
print (d.dtypes)
d.head()

Spot Time           object
Spot Date           object
Spot Price          object
Expiration Date     object
Symbol              object
Intrinsic Value    float64
Last               float64
Chg                float64
Bid                float64
Ask                float64
Volume               int64
Open Int             int64
Delta              float64
Gamma              float64
Vega               float64
Theta              float64
Contract High      float64
Strike             float64
dtype: object


,Spot Time,Spot Date,Spot Price,Expiration Date,Symbol,Intrinsic Value,Last,Chg,Bid,Ask,Volume,Open Int,Delta,Gamma,Vega,Theta,Contract High,Strike
0,04:00:00 PM,09/25/20,$108.22,10/02/20,AAPL201002C103.75,4.37,6.00,0.50,5.90,6.05,1051,523,0.7125,0.0398,0.0547,-0.1854,94.55,103.75
1,04:00:00 PM,09/26/20,$108.22,10/02/20,AAPL201002C105,3.12,5.10,0.35,5.00,5.25,3111,3513,0.6612,0.0434,0.0585,-0.1967,27.12,105.00
2,04:00:00 PM,09/27/20,$108.22,10/02/20,AAPL201002C106.25,1.87,4.33,0.24,4.20,4.40,2446,1574,0.6052,0.0463,0.0619,-0.2048,29.55,106.25
3,04:00:00 PM,09/28/20,$108.22,10/02/20,AAPL201002C107.5,0.62,3.60,0.20,3.55,3.70,10299,7077,0.5461,0.0487,0.0636,-0.2068,27.54,107.50
4,04:00:00 PM,09/29/20,$108.22,10/02/20,AAPL201002C108.75,0.00,2.92,0.11,2.88,3.00,9176,4146,0.4847,0.0495,0.0639,-0.2059,27.15,108.75


In [128]:
#Parse dates
d["SpotDt"] = pd.to_datetime(d['Spot Date'], format="%m/%d/%y")
d["ExpDt"] = pd.to_datetime(d['Expiration Date'], format="%m/%d/%y")
#Override Spot Date
currDate = datetime.datetime.strptime("09/25/20", "%m/%d/%y")
d["SpotDt"] = currDate
#Add days to expiration
d['Days2Exp'] = d['ExpDt'] - d['SpotDt']
d['Spot Price'] = 108.22
d['PriceDiff'] = np.subtract (d['Strike'] , 108.22)

#test = d[['Delta', 'Gamma', 'Vega', 'Theta', 'Spot Price', 'Strike', 'Expiration Date', 'Spot Date']]

In [129]:
#Change all Spot dates to corrected Date 9/25/2020
#d['SpotDt']= np.full_like (np.ndarray([1,d.shape [0]]), fill_value = pd.to_datetime("09/25/2020", format="%m/%d/%y") )
#d['SpotDt'] = '09/25/2020'
print (d.head())

     Spot Time Spot Date  Spot Price  ... PriceDiff    ImpVal   HistVal
0  04:00:00 PM  09/25/20      108.22  ...     -4.47 -3.184875 -3.184875
1  04:00:00 PM  09/26/20      108.22  ...     -3.22 -2.129064 -2.129064
2  04:00:00 PM  09/27/20      108.22  ...     -1.97 -1.192244 -1.192244
3  04:00:00 PM  09/28/20      108.22  ...     -0.72 -0.393192 -0.393192
4  04:00:00 PM  09/29/20      108.22  ...      0.53  0.256891  0.256891

[5 rows x 24 columns]


## Use HV30  and IV30 for options
# As of 9/27/2020 HV30 is 50.03 and IV30 is 46.02

In [130]:
hv30 = 50.03
iv30 = 46.02
spot_price = 108.22
iv_dis = 100* ( iv30/hv30  - 1.0)
print (iv_dis)

-8.015190885468716


In [131]:
##Test Area
np.ndarray([1,10])
d['Days2Exp'].dt.days

np.sqrt(d['Days2Exp'].dt.days) 

0      2.645751
1      2.645751
2      2.645751
3      2.645751
4      2.645751
         ...   
115    9.165151
116    9.165151
117    9.165151
118    9.165151
119    9.165151
Name: Days2Exp, Length: 120, dtype: float64

In [132]:
#Compute Del-gamma-vega-theta values

d['ImpVal'] = np.multiply(d['Delta'], d['Spot Price'])  
+ np.multiply (d['Gamma'], np.sqrt(d['Spot Price'])) * 0.5
+ np.multiply (d['Theta'] , d['Days2Exp']) 
+ d['Vega'] * iv30 * np.sqrt(d['Days2Exp'].dt.days / 30.0) 



d['HistVal'] = np.multiply (d['Delta'], d['Spot Price']) 
+ np.multiply (d['Gamma'], np.sqrt(d['Spot Price'])) *0.5
+ np.multiply (d['Theta'], d['Days2Exp'])
+ np.multiply (d['Vega'] ,  hv30 * np.sqrt(d['Days2Exp'].dt.days / 30.0) )

0       1.321923
1       1.413757
2       1.495924
3       1.537008
4       1.544258
         ...    
115    17.404598
116    17.379484
117    17.195308
118    16.860443
119    16.383261
Length: 120, dtype: float64

In [133]:
d.head()

,Spot Time,Spot Date,Spot Price,Expiration Date,Symbol,Intrinsic Value,Last,Chg,Bid,Ask,Volume,Open Int,Delta,Gamma,Vega,Theta,Contract High,Strike,SpotDt,ExpDt,Days2Exp,PriceDiff,ImpVal,HistVal
0,04:00:00 PM,09/25/20,108.22,10/02/20,AAPL201002C103.75,4.37,6.00,0.50,5.90,6.05,1051,523,0.7125,0.0398,0.0547,-0.1854,94.55,103.75,2020-09-25,2020-10-02,7 days,-4.47,77.106750,77.106750
1,04:00:00 PM,09/26/20,108.22,10/02/20,AAPL201002C105,3.12,5.10,0.35,5.00,5.25,3111,3513,0.6612,0.0434,0.0585,-0.1967,27.12,105.00,2020-09-25,2020-10-02,7 days,-3.22,71.555064,71.555064
2,04:00:00 PM,09/27/20,108.22,10/02/20,AAPL201002C106.25,1.87,4.33,0.24,4.20,4.40,2446,1574,0.6052,0.0463,0.0619,-0.2048,29.55,106.25,2020-09-25,2020-10-02,7 days,-1.97,65.494744,65.494744
3,04:00:00 PM,09/28/20,108.22,10/02/20,AAPL201002C107.5,0.62,3.60,0.20,3.55,3.70,10299,7077,0.5461,0.0487,0.0636,-0.2068,27.54,107.50,2020-09-25,2020-10-02,7 days,-0.72,59.098942,59.098942
4,04:00:00 PM,09/29/20,108.22,10/02/20,AAPL201002C108.75,0.00,2.92,0.11,2.88,3.00,9176,4146,0.4847,0.0495,0.0639,-0.2059,27.15,108.75,2020-09-25,2020-10-02,7 days,0.53,52.454234,52.454234
